# Diet Lab
Based on chapter 2 of the AMPL book and section 6.2 of *Decision Making, Models and Algorithms: A First Course* by Saul I. Gass.

**Objectives**
- Introduce an historically important linear program
- Think about solving linear programs and tweaking solutions
- Demonstrate the idea of sensitivity analysis in linear programming

**Brief description:** In this lab we will consider one of the most famous (and one of the earliest) appli-
cations of linear programming — the diet problem.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

In [1]:
# imports -- don't forget to run this cell
import pandas as pd
from IPython.display import Image
from ortools.linear_solver import pywraplp as OR

## Part 1: The Diet Problem

Suppose that you have a choice of two cereals for breakfast: Krunchies (K) or Crispies (C). Breakfast is the
most important meal of the day, so you want to make sure you get sufficient Thiamine, Niacin, and caloric
intake for breakfast. Being a college student, you want to do so as cheaply as possible – taste gets thrown
into the wind! Price and nutritional info for these two cereals is summarized in the two tables below:

In [2]:
# Create the two tables
nutrients = pd.DataFrame([{'Name':'Thiamine', 'Min':1},
                          {'Name':'Niacin', 'Min':5},
                          {'Name':'Calories', 'Min':400}]).set_index('Name')
cereals = pd.DataFrame([{'Name':'Krunchies', 'Cost':3.80, 'Thiamine':0.1, 'Niacin':1.0, 'Calories':110},
                        {'Name':'Crispies', 'Cost':4.20, 'Thiamine':0.25, 'Niacin':0.25, 'Calories':120}])
cereals = cereals.set_index('Name')

In [3]:
display(cereals)
display(nutrients)

,Cost,Thiamine,Niacin,Calories
Name,,,,
Krunchies,3.8,0.10,1.00,110
Crispies,4.2,0.25,0.25,120


,Min
Name,
Thiamine,1
Niacin,5
Calories,400


**Q1:** Suppose you just ate Krunchies. How many ounces of Krunchies would you need to eat to satisfy the
three nutritional requirements? How much would this cost?

**A:** <font color='blue'>10 ounces to meet Thiamine requirement. 5 ounces to meet Niacin requirement. 3.636 ounces to satisfy Calories requirement. Hence, 10 ounces to satisfy all three. </font>

**Q2:** Suppose you just ate Crispies. How many ounces of Crispies would you need to eat to satisfy the three
nutritional requirements? How much would this cost?

**A:** <font color='blue'>4 ounces to meet Thiamine requirement. 20 ounces to meet Niacin requirement. 3.333 ounces to satisfy Calories requirement. Hence, 20 ounces to satisfy all three. </font>

**Q3:** Now let’s write an optimization problem to model this problem. Let $K$ be a decision variable for the amount of Krunchies you eat and $C$ be a decision variable for the amount of Crispies you eat. Write an objective function, encoding that you minimize total cost (as a function of $K$ and $C$).

**A:** <font color='blue'> $\min 3.8K + 4.2C$ </font>

**Q4:** Write three constraints: one enforcing that you get at least 1 mg of Thiamine, one enforcing that you
get at least 5 mg of Niacin, and one enforcing that you get at least 400 calories. Also write constraints
that $K$ and $C$ are nonnegative.

**A: <font color='blue'> $$0.1K + 0.25C \geq 1$$ 
$$1K + 0.25C \geq 5$$ 
$$110K + 120C \geq 400$$** </font>

**Q5:** Implement this model in OR-Tools. The basic structure has been given to you.

In [4]:
# define the model
m = OR.Solver('diet', OR.Solver.CLP_LINEAR_PROGRAMMING);

# TODO: Define the decision variables.
# m.NumVar(lower_bound, upper_bound, name)
# If you want to use infinity as a bound, you can use m.infinity()

# SOLUTION 
K = m.NumVar(0, m.infinity(), 'K');
C = m.NumVar(0, m.infinity(), 'C');

# TODO: Define the objective function.
# m.Minimize() or m.Maximize()

# SOLUTION 
m.Minimize(3.8*K + 4.2*C);

# TODO: Define the constraints.
# m.Add()

# SOLUTION 
m.Add(0.1*K + 0.25*C >= 1);
m.Add(1*K + 0.25*C >= 5);
m.Add(110*K + 120*C >= 400);

# Solve and print solution
m.Solve()
print('Objective =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective = 26.22222222222222
Solution:
K : 4.444444444444445
C : 2.2222222222222223


## Part 2: Generalizing the Diet Problem

We now know that if we expect to live a long and healthy life, we must have a well-balanced diet. Too much fat or sodium in our diet can lead to serious health problems. Similarly, diets deficient in essential vitamins and minerals should be avoided. 

This part of this lab is aimed at helping you to formulate this problem as a linear programming problem. We can view the diet problem in the following way. We are given a variety of foods that we could buy to achieve a balanced diet. For example, we might consider diet consisting of 2% milk, spaghetti (with sauce), peanut butter, wheat bread, tomato soup, and bagels. To make things simpler, we will specify the variables of this linear programming formulation. We will use $x$_$\textit{(food-type)}$ to specify the number of daily servings of food-type that you are willing to consume. First of all, we wish to write constraints that capture whether one can satisfy certain daily requirements with just these foods.

**Q:** Write constraints that ensure that the diet specifies that we eat at most 10 servings/day of each food-type (We can call this the boredom constraint).

**A:** <font color='blue'> Let $F$ be the set of foods. We have $x_i \leq 10$ for $i \in F$. </font>

Next consider the total number of calories consumed. A proper diet requires that you consume between 2000 and 2250 calories per day. 

**Q:** Write two constraints that ensure that the diet specifies that we eat an appropriate number of total calories. To write this constraint you need to know how many calories are in one serving of each of the food-types. Let $a_{ij}$ be the amount of nutrient $j$ in food $i$.

**A:**  <font color='blue'> $\sum_{i\in F} a_{ij}x_i \geq 2000$ and $\sum_{i\in F} a_{ij}x_i \leq 2250$ where $j$ = calories.</font>

Of course, we could repeat this sort of constraint with any number of nutrient requirement, such as cholesterol, fat, sodium, dietary fiber, carbohydrates, protein, vitamin A, vitamin C, calcium, and iron.

**Q:** The objective function is to minimize the cost of each day’s diet. Express this objective function in terms of the decision variables, given that $c_i$ is the cost of one serving of food $i$.

**A:** <font color='blue'> $\min \sum_{i\in F} c_ix_i$ </font>

**Q:** What does it mean if your linear program for this diet problem is infeasible?

**A:**

**Q:** How might you correct this?
    
**A:**

Let's use OR-Tools to implement this generalized diet problem model!

**Q:** Complete the model below.

In [5]:
def Diet(foods, nutrients, integer=False):
    """Classic Diet Problem Model.
    
    Args:
        foods (pd.DataFrame): Foods with cost per serving and nutrients per serving.
        nutrients (pd.DataFrame): Nutrients with min and max bounds.
    """
    FOODS = list(foods.index)                                 # foods
    NUTRIENTS = list(nutrients.index)                         # nutrients
    c = foods['Cost'].to_dict()                               # cost per serving of food 
    f_min = foods['Min'].to_dict()                            # lower bound of food serving
    f_max = foods['Max'].to_dict()                            # upper bound of food serving
    n_min = nutrients['Min'].to_dict()                        # lower bound of nutrient
    n_max = nutrients['Max'].to_dict()                        # upper bound of nutrient  
    a = foods[list(nutrients.index)].transpose().to_dict()    # amt of nutrients per serving of food
    
    # define model
    if integer:
        m = OR.Solver('diet', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    else:
        m = OR.Solver('diet', OR.Solver.CLP_LINEAR_PROGRAMMING)
        
    # decision variables
    x = {}    
    for i in FOODS:
        if integer:
            x[i] = m.IntVar(0, m.infinity(), 'x_%s' % (i)) 
        else:
            x[i] = m.NumVar(0, m.infinity(), 'x_%s' % (i)) 
        
    # define objective function here
    m.Minimize(sum(c[i]*x[i] for i in FOODS))
    
    # enforce lower and upper bound on food servings
    for i in FOODS:
        m.Add(x[i] >= f_min[i], name='lb_%s' % (i))
        m.Add(x[i] <= f_max[i], name='ub_%s' % (i))
    
    # enforce lower and upper bound on nutrients 
    for j in NUTRIENTS:
        m.Add(sum(a[i][j]*x[i] for i in FOODS) >= n_min[j], name='lb_%s' % (j))
        m.Add(sum(a[i][j]*x[i] for i in FOODS) <= n_max[j], name='ub_%s' % (j))
        
    return (m, x)  # return the model and the decision variables

In [6]:
# You do not need to do anything with this cell but make sure you run it!
def solve(m):
    """Used to solve a model m."""
    m.Solve()
    
    print('Objective =', m.Objective().Value())    
    return {var.name() : var.solution_value() for var in m.variables()}

Use this model to solve the cereal model. Add question about how to modify input to do this.

In [7]:
display(cereals)
display(nutrients)

,Cost,Thiamine,Niacin,Calories
Name,,,,
Krunchies,3.8,0.10,1.00,110
Crispies,4.2,0.25,0.25,120


,Min
Name,
Thiamine,1
Niacin,5
Calories,400


In [8]:
cereals['Min'] = 0 
cereals['Max'] = float('inf') 
nutrients['Max'] = float('inf') 

In [9]:
m,x = Diet(cereals, nutrients)
solve(m)

Objective = 26.22222222222222


{'x_Krunchies': 4.444444444444445, 'x_Crispies': 2.2222222222222223}

## Part 3: Solving the Diet Problem

You will use an interactive program for the diet problem which was developed at Argonne National Labs. Open a browser and go to the URL,
   http://www.neos-guide.org/NEOS/index.php/Diet_Problem_Demo
Click on the link to the “case study” in the first line of this wiki page. Read the entry on the history of the diet problem first.

**Q:** Now return to the wiki page, and submit a selection of foods for your diet. In particular, try clicking on the set of food-types that were mentioned above as a starter set of food-types. You will see that there is no feasible solution. Take a look at the specifics of the nutrient requirements and the nutrient contents, and try to understand why there is no feasible solution. What do think caused it to be infeasible? (I don’t know of a simple answer to this.)

**A:**

**Q:** Now consider a much wider set of food-types. First try the “Default” set of foods. What is the solution found by the algorithm?

**A:**

**Q:** How many different food-types are included in this diet? Check this carefully. Is this surprising to you? How much does this diet cost?

**A:**

**Q:** Now view the summary of the “dual and constraint information”. These tables indicate how much of each nutrient you will actually consume. For which nutrients are you at your upper limit? Such a constraint is said to be tight.

**A:**

**Q:** For which are you at your lower limit? (That is, which lower limits are tight?)

**A:**

**Q:** Are any of the minimum and maximum number of serving constraints tight?

**A:**

**Q:** You should have included carbohydrates in the list of nutrients that is at its upper limit in your diet. Make a note of the dual cost for the upper bound on carbohydrates. Suppose you increase the allowance for carbohydrates to 305 grams. (This requires returning to the food selection table, and checking that you want to edit the requirements.) How does the optimal diet change? By how much does its cost change?

**A:**

**Q:** Now return to the dual and constraint information. The “dual cost” for carbohydrates is given there. This information can be used in the following way. Suppose that a dual cost for a particular nutrient is −c , (i.e., it is negative). Now suppose that we may have one more unit of this nutrient in our diet, and when we find the optimal solution for this modified data, the foods that make up the optimal diet are unchanged. First of all, how does the cost of the optimal solution to the new data compare to the optimal solution to the original data? Why?

**A:**

**Q:** In fact, the dual cost tells how much cheaper the new solution will be: it will be c units cheaper. Use the dual cost for carbohydrates for the original (default) data to double check the change in cost for the optimum when you change the carbohydrate requirement from 300 to 305.

**A:**

**Q:** Now allow up to 320 grams of carbohydrates in your diet. How does this change things? Does the analogous calculation to the one that you just did still work? Why do you think that this is?

**A:**

**Q:** Now go back to the default data, its optimal solution, and the dual and constraint information for it. Which constraint do you think it would be most profitable to violate by one unit? Can you give an intuitive explanation of this?
    
**A:**    

**Q:** As you can tell from the output, the optimal solution largely calls for non-integral numbers of servings of the various food-types. Why might this be problematic? Is there any explanation of the set-up for the problem that would allow us to consider such fractional solutions?

**A:**

**Q:** Suppose that you are buying just one day’s worth of food, and you must buy an integral number of servings of each food-type. Can you still use this output to construct a diet? Is this new selection an optimal diet subject to the restriction that the number of servings bought be integral?

**A:**

**Q:** Without worrying about how to compute the optimal integer solution, how does the cost of the optimal integer solution compare to the cost of our linear program’s optimal solution? 

**A:**

**Q:** As a bit of useful terminology, a linear program in which the variables are further constrained to take on integer values is called an integer linear program, or sometimes just an integer program, for short. If we remove the requirement that the variables take integer values, then these constraints are said to be relaxed, and we have the linear programming relaxation of this integer program. For any minimization integer linear program, how does the optimal value of the integer program compare to the optimal value of its linear relaxation? (Compare just the values, not the optimal solutions themselves.)

**A:**

**Q:** What do you think about eating the diet proposed by the output? How might you modify the constraints to get something more to your liking?

**A:**

**Q:** Now go back to the input selection menu, and try to find a cheaper diet. How cheap a diet can you find? What is this diet?
    
**A:**

## Model And Data

In [10]:
# foods = pd.read_csv('small_diet.csv').set_index('Name')
# foods['Min'] = 0
# foods['Max'] = float('inf')
# nutrients = pd.DataFrame(index=['Vitamin A', 'Vitamin C',
#                                 'Vitamin B1', 'Vitamin B2'])
# nutrients['Min']= 700 # 100% over an entire week (7 days)
# nutrients['Max']= float('inf')

In [11]:
foods = pd.read_csv('neos_foods.csv', index_col=0)
nutrients = pd.read_csv('neos_nutrients.csv', index_col=0)

In [12]:
m.Solve()
for i in m.constraints():
    name = i.name()
    dual_value = i.dual_value()
    if dual_value != 0:
        print(i.name(), i.dual_value())

lb_Thiamine 14.444444444444446
lb_Niacin 2.355555555555555


## 3 A More Robust Diet

Let’s now expand our diet. Instead of looking at just breakfast, we’ll allow ourselves to eat more food. The
food options, together with the decision variable we’ll use, are:
- Beef ($X_{beef}$ )
- Chicken ($X_{chk}$)
- Fish ($X_{fish}$)
- Ham ($X_{ham}$)
- Mac & cheese ($X_{mch}$)
- Meat loaf ($X_{mtl}$)
- Spaghetti ($X_{spg}$)
- Turkey ($X_{tur}$)

As before, we’ll want to minimize cost. This time, we’ll want to meet nutritional requirements for vitamins
$A, C, B1$, and $B2$.

Costs (in dollars) and nutritional information are shown below. All costs/nutritional requirements are
per package. The values for nutritional requirements are percents. E.g. one package of beef provides 60% of
the daily requirement for vitamin A. In a day, you need 100% of your recommended amount of Vitamin A.

In [13]:
pd.read_csv('small_diet.csv')

,Name,Cost,Vitamin A,Vitamin C,Vitamin B1,Vitamin B2
0,Beef,3.19,60,20,10,15
1,Chk,2.59,8,0,20,20
2,Fish,2.59,8,10,15,10
3,Ham,2.89,40,40,35,10
4,Mch,1.89,15,35,15,15
5,Mtl,1.99,70,30,15,15
6,Spg,1.99,25,50,25,15
7,Tur,2.49,60,20,15,10


Let’s consider the problem of minimizing cost of a diet over an entire week.

**Q:** Write out the objective function for this problem (which is to minimize cost).

**A:**

**Q:** Write out the constraint indicating that you get enough Vitamin A in an entire week. Be careful about
the right hand side: what percent do you need over an entire week?

**A:**

**Q:** What solution does it give? How much does it cost, which food(s) do we eat, and how much of those food(s) do we eat?

**A:**

**Q:** Does the diet seem reasonable?

**A:**

Let’s add a constraint capping the maximum amount of mac and cheese we eat at 20 servings. (some explanation of how to do this ...)

**Q:** What is the new solution? How much more expensive is it? Does it seem healthier?

**A:**

**Q:** Suppose you wanted to add in 3 more food options (soylent, snickers, and coffee) and 10 more con-
straints (on vitamins B3, B5, B6, B9, B12, D, E, and K; on Sodium; and on the most important vitamin of all, caffeine). Or suppose that I asked you to instead do this problem with bread, meat,
potatoes, cabbage, milk, and gelatin. Write 2-3 sentences actually explaining what you’d have to do
in each of these two settings. Also briefly describe how tedious of a process it would be to edit this
model. Colorful (but not vulgar) language, metaphors, and famous quotes are all appropriate.

**A:**